In [6]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os

In [ ]:
# Get the current working directory
baseFolder = os.getcwd()

# Define the folder containing the images
folder = os.path.join(baseFolder, "photos", "largeBubbles")

# Read the image in grayscale
img = cv.imread(os.path.join(folder, "1.png"),0)